# 任务3重构

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-07-14  

以快照数据为时间节点重新梳理任务

构建流程：

> 2018年2月的快照数据为训练集  （469,134条）
> 2020年6月的快照数据 （476,011条） - 2018年2月份快照数据中出现的序列数据（469,134条）  差集作为测试集 (8033条，其中7开头的数据 3619 条)

## 1. 导入必要的包

In [39]:
import numpy as np
import pandas as pd
import sys
import os
from tqdm import tqdm
sys.path.append("../../tools/")
import ucTools
import funclib
from pandarallel import pandarallel
pandarallel.initialize()
sys.path.append("../../")
import benchmark_config as cfg
%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. 定义函数功能

In [40]:
def ec_appears_statistic(data):
    """
    统计EC号出现的频次
    返回[EC号，出现频次] 的DataFrame
    """
    eclist = funclib.get_distinct_ec(data)
    resdf = pd.DataFrame(eclist)
    resdf.index = eclist
    resdf[0]=0
    
    for item in data:
        ecarray = item.split(',')
        for subitem in ecarray:
            resdf[resdf.index==subitem.strip()] +=1
            
    return resdf

## 3. 读取数据

In [41]:
# 2018 快照
snap2018 = pd.read_csv('../../data/201802/sprot_full.tsv', sep='\t', header=None, names=funclib.table_head)
snap2018 = snap2018.sort_values(by='id').reset_index(drop=True) 
snap2018.drop_duplicates(subset=['seq'], keep='first', inplace =True)
snap2018.reset_index(drop=True, inplace=True)



#2020 快照
snap2020 = pd.read_csv('../../data/202006/sprot_full.tsv', sep='\t', header=None, names=funclib.table_head)
snap2020 = snap2020.sort_values(by='id').reset_index(drop=True)
snap2020.drop_duplicates(subset=['seq'], keep='first', inplace =True)
snap2020.reset_index(drop=True, inplace=True)

#将序列去空格，转化成大写字母
snap2018['seq'] = snap2018.seq.apply(lambda x: x.strip().upper())
snap2020['seq'] = snap2020.seq.apply(lambda x: x.strip().upper())

#处理EC号后边为.的数据
snap2018['ec_number']=snap2018.parallel_apply(lambda x: x.ec_number+'-' if x.ec_number.endswith('.') else x.ec_number, axis=1)
snap2020['ec_number']=snap2020.parallel_apply(lambda x: x.ec_number+'-' if x.ec_number.endswith('.') else x.ec_number, axis=1)

In [5]:
# #展开后的快照数据
# snap2018M = pd.read_csv('../../data/201802/snap2018_siglelineec.tsv', sep='\t', header=0)
# snap2020M = pd.read_csv('../../data/202006/snap2020_siglelineec.tsv', sep='\t', header=0)

In [13]:
# snap2018M['ec_number']=snap2018M.ec_number.apply(lambda x:x.replace('_', '-'))
# snap2020M['ec_number']=snap2020M.ec_number.apply(lambda x:x.replace('_', '-'))

# snap2018M.to_csv('../../data/201802/snap2018_siglelineec.tsv', sep='\t', index=None)
# snap2020M.to_csv('../../data/202006/snap2020_siglelineec.tsv', sep='\t', index=None)

In [8]:
# 写入数据库中
uctools =  ucTools.ucTools('172.16.25.20')
cnx_mimic = uctools.db_conn()
uctools.saveToDB(snap2018M, 'tb_snap_2018', cnx_mimic)
uctools.saveToDB(snap2020M, 'tb_snap_2020', cnx_mimic)

In [6]:
snap2018

,id,name,isemzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,A0A023GPI8,LECA_CANBL,False,False,0,-,0,2016-05-11,2014-06-11,2017-11-22,ADTIVAVELDTYPNTDIGDPSYPHIGIDIKSVRSKKTAKWNMQNGK...,236
1,A0A023GPJ0,CDII_ENTCC,False,False,0,-,0,2015-03-04,2015-03-04,2017-07-05,MFGIFSKGEPVSMEGELVQPSSIVINDYEEELHLPLSYWDIKDYKN...,143
2,A0A023PXA5,YA19A_YEAST,False,False,0,-,0,2014-11-26,2014-07-09,2018-02-28,MLLSELVATASSLPYTAISIHNNCRVPAARHIHHGCRYFHGPPVMH...,189
3,A0A023PXB0,YA019_YEAST,False,False,0,-,0,2014-11-26,2014-07-09,2018-01-31,MFINGFVNYPVRTPPNDLLQVVLHGFLRCPLDGSQVDSIGIGHTVH...,110
4,A0A023PXB5,IRC2_YEAST,False,False,0,-,0,2014-11-26,2014-07-09,2018-01-31,MFALIISSKGKTSGFFFNSSFSSSALVGIAPLTAYSALVTPVFKSF...,102
...,...,...,...,...,...,...,...,...,...,...,...,...
469124,X5IWS1,CO17J_CONGE,False,False,0,-,0,2017-02-15,2017-02-15,2018-01-31,MKLTCVVIVAALLLTACQLITALDCGGTQKHRALRSTIKLSLLRQH...,82
469125,X5JA13,SECAA_ARATH,False,False,0,-,0,2017-01-18,2014-06-11,2018-01-31,MTEGIRARGPRSSSVNSVPLILDIEDFKGDFSFDALFGNLVNDLLP...,825
469126,X5JB51,SECAB_ARATH,False,False,0,-,0,2017-01-18,2014-06-11,2018-02-28,MTERIRARGPRSSSVNSVPLILDIEDFKGDFSFDALFGNLVNDLLP...,829
469127,X5M5N0,WNK_CAEEL,True,False,1,2.7.11.1,4,2015-06-24,2014-06-11,2018-02-28,MPDSITNGGRPPAPPSSVSSTTASTTGNFGTRRRLVNRIKKVDELH...,1850


## 4. 计算-统计数据

In [5]:
# 计算差集
list_subtraction = list(set(snap2020.seq) - set(snap2018.seq))
data_subtraction = snap2020[snap2020.seq.isin(list_subtraction)]
data_subtraction.reset_index(drop=True, inplace=True)

# 计算保存EC列表
ec_2018 = funclib.get_distinct_ec(snap2018.ec_number)
ec_2020 = funclib.get_distinct_ec(snap2020.ec_number)
ec_added = sorted(list(set(ec_2020) - set(ec_2018)))
ec_notappear = sorted(list(set(ec_2018) - set(ec_2020)))
ec_common = sorted(set(ec_2018) & set(ec_2020))

In [10]:
ec_counts_2018 =ec_appears_statistic(snap2018.ec_number)
ec_counts_2020 =ec_appears_statistic(snap2020.ec_number)

## 5.保存数据  

In [203]:
# 保存EC列表
pd.DataFrame(ec_2018, columns=['ec_number']).to_csv('../../results/task3r/ec2018.tsv', sep='\t')
pd.DataFrame(ec_2020, columns=['ec_number']).to_csv('../../results/task3r/ec2020.tsv', sep='\t')
pd.DataFrame(ec_added, columns=['ec_number']).to_csv('../../results/task3r/ecadded.tsv', sep='\t')
pd.DataFrame(ec_notappear, columns=['ec_number']).to_csv('../../results/task3r/ecnotappear.tsv', sep='\t')
pd.DataFrame(ec_common, columns=['ec_number']).to_csv('../../results/task3r/eccommon.tsv', sep='\t')


In [240]:
ec_counts_2018.to_csv('../../results/task3r/eccounts_2018.tsv', sep='\t')
ec_counts_2020.to_csv('../../results/task3r/eccounts_2020.tsv', sep='\t')

In [4]:
pd.DataFrame(columns=snap2018.columns.values)

,id,name,isemzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength


# SECTION 3 unirep

In [42]:
uniprep = pd.read_feather('../../data/sprot_unirep_bank.feather')

In [30]:
from jax_unirep import get_reps

def getunirep(enzyme_noemzyme, step):
    unirep_res = []
    counter = 1
    for i in tqdm(range(0, len(enzyme_noemzyme), step)):
        train_h_avg, train_h_final, train_c_final= get_reps(list(enzyme_noemzyme.seq[i:i+step]))
        checkpoint = np.hstack((np.array(enzyme_noemzyme[i:i+step]),train_h_final))
        
        if counter == 1:
            unirep_res = np.array(checkpoint)
        else:
            unirep_res = np.concatenate((unirep_res,checkpoint))

        np.save(r'/tmp/task_unirep_'+str(counter)+'.tsv', checkpoint)

        if len(train_h_final) != step:
            print('length not match')
        counter += 1
    
    return unirep_res

In [26]:
need_unirep = snap2018[~snap2018.seq.isin(uniprep.seq)]

In [6]:
uniprep

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900,seq
0,0.000455,-0.021951,0.027487,-0.000919,-0.228522,0.004351,-0.140352,-0.000167,-0.001996,0.658748,...,0.119389,-0.019647,-0.016905,-0.532317,0.008659,-0.011296,0.001098,0.794472,0.210124,MDPNCSCSTGGSCTCTSSCACKNCKCTSCKKSCCSCCPVGCSKCAQ...
1,-0.001693,0.002171,-0.007579,-0.027573,-0.161033,-0.030274,-0.644947,-0.000623,-0.011457,0.167647,...,0.003742,-0.048072,0.030513,-0.260276,-0.070536,0.002627,-0.002706,0.682809,0.455698,AASPQKRAASPRKSPKKSPRKSPKKKSPRKRKARSAAHPPVIDMIT...
2,0.003249,-0.027980,0.021816,-0.031098,-0.452770,0.153213,-0.164409,-0.003949,-0.000905,0.202440,...,-0.181347,0.033731,0.023869,0.357923,0.098604,-0.009344,0.012374,-0.027197,0.109520,MRLAGWGLRWAIALLIAVGEAAVEDNCGRNEFQCQDGKCISYKWVC...
3,0.001718,0.191248,0.015037,-0.013877,-0.284253,0.006640,-0.425640,0.011292,-0.006632,0.157273,...,-0.098609,0.091042,0.331697,-0.060142,-0.067148,-0.039508,-0.044459,-0.043506,0.125468,MGKNGSLCCFSLLLLLLLAGLASGHQVLFQGFNWESWKQSGGWYNM...
4,-0.499300,-0.029771,0.034598,-0.001920,-0.011458,-0.006578,-0.713303,-0.000076,-0.002641,0.020854,...,-0.004334,-0.019177,0.002985,-0.078540,-0.003586,0.119747,-0.051190,0.989171,-0.141614,MSDSAVATSASPVAAPPATVEKKVVQKKASGSAGTKAKKASATPSH...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500817,0.000861,-0.037308,0.177682,-0.002576,-0.416290,0.005963,-0.049527,-0.073934,-0.008551,-0.046575,...,0.029201,-0.378917,0.573398,-0.082814,0.020948,0.488781,-0.001050,-0.173049,0.011326,MEVVEVLHMNGGNGDSSYANNSLVQQKVILMTKPITEQAMIDLYSS...
500818,0.000780,-0.052708,0.138153,-0.002266,-0.426327,-0.001983,0.232379,-0.072236,-0.003973,0.045996,...,-0.000318,-0.341042,0.506305,-0.080408,0.024440,0.467743,-0.000233,-0.145555,0.007762,MEVVEVLHMNGGNGDSSYANNSLVQQKVILMTKPITEQAMIDLYSS...
500819,0.002115,-0.014731,0.088683,-0.007679,0.131181,0.012399,-0.195394,-0.005713,-0.001974,0.299056,...,0.047640,-0.059279,0.021625,-0.090096,-0.047780,0.148048,-0.010095,0.101847,0.158366,MTTEQEFEKVELTADGGVIKTILKKGDEGEENIPKKGNEVTVHYVG...
500820,0.001798,-0.013006,0.027276,-0.012069,-0.326636,0.018492,-0.159893,-0.013038,-0.001523,-0.092576,...,-0.322253,0.038835,-0.006214,-0.380154,0.009995,0.144725,0.011319,-0.086592,0.004265,MKIANNTVVTEFILLGLTQSQDIQLLVFVLILIFYLIILPGNFLII...


# SECTION 4 构建训练测试数据

## 4.1 训练数据

In [45]:
DATAROOT= '/home/shizhenkun/codebase/BioUniprot/data/benchmark/data/'

# 写入「是否是酶」训练数据
funclib.table2fasta(snap2018, DATAROOT+'train.fasta')

Write finished


In [46]:
DATAROOT= '/home/shizhenkun/codebase/BioUniprot/data/benchmark/data/'

# 拼合训练数据
train_rep=snap2018.merge(uniprep, on='seq', how='left')

# # 写入酶训练数据

train_rep = train_rep.iloc[:, np.r_[0,2,4:7,10:1912]]
train_rep = train_rep[(train_rep.ec_specific_level>=0) | (~train_rep.isemzyme)]
train_rep.reset_index(drop=True, inplace=True)
train_rep.to_feather(cfg.TRAIN_FEATURE)

In [58]:
train_rep

,id,isemzyme,functionCounts,ec_number,ec_specific_level,seq,seqlength,f1,f2,f3,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,A0A023GPI8,False,0,-,0,ADTIVAVELDTYPNTDIGDPSYPHIGIDIKSVRSKKTAKWNMQNGK...,236,0.000115,0.105367,0.037413,...,0.052682,-0.020702,0.081852,0.008791,-0.036727,-0.027552,-0.254128,0.020778,0.140904,0.140635
1,A0A023GPJ0,False,0,-,0,MFGIFSKGEPVSMEGELVQPSSIVINDYEEELHLPLSYWDIKDYKN...,143,0.001688,-0.017209,0.029518,...,0.040570,-0.031992,-0.174783,0.000952,-0.136715,0.042773,0.132279,0.015899,0.152930,-0.003412
2,A0A023PXA5,False,0,-,0,MLLSELVATASSLPYTAISIHNNCRVPAARHIHHGCRYFHGPPVMH...,189,0.003066,-0.010453,0.013398,...,0.014996,0.034800,-0.221564,0.014209,-0.019194,-0.006493,0.121753,0.040914,0.143036,-0.024102
3,A0A023PXB0,False,0,-,0,MFINGFVNYPVRTPPNDLLQVVLHGFLRCPLDGSQVDSIGIGHTVH...,110,0.016568,-0.021448,0.025598,...,0.081018,-0.162543,-0.121671,0.006464,-0.045551,-0.033775,0.038735,-0.108828,0.042148,0.042277
4,A0A023PXB5,False,0,-,0,MFALIISSKGKTSGFFFNSSFSSSALVGIAPLTAYSALVTPVFKSF...,102,0.002214,-0.008754,0.012546,...,0.000570,-0.177887,-0.351524,0.017807,0.052179,0.050745,0.439712,0.071341,0.324028,-0.041494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469124,X5IWS1,False,0,-,0,MKLTCVVIVAALLLTACQLITALDCGGTQKHRALRSTIKLSLLRQH...,82,0.000362,-0.116292,0.106362,...,-0.043532,-0.140887,0.032296,-0.025618,-0.397847,0.008253,-0.271723,-0.034782,-0.209635,0.154475
469125,X5JA13,False,0,-,0,MTEGIRARGPRSSSVNSVPLILDIEDFKGDFSFDALFGNLVNDLLP...,825,0.000235,0.023418,0.002429,...,0.000126,0.023057,-0.052336,0.024291,-0.072905,-0.127662,0.120465,0.004537,-0.033176,0.015282
469126,X5JB51,False,0,-,0,MTERIRARGPRSSSVNSVPLILDIEDFKGDFSFDALFGNLVNDLLP...,829,0.000265,0.024664,0.002740,...,0.000863,0.025310,-0.059424,0.025009,-0.076028,-0.131405,0.114073,0.005262,-0.034246,0.000684
469127,X5M5N0,True,1,2.7.11.1,4,MPDSITNGGRPPAPPSSVSSTTASTTGNFGTRRRLVNRIKKVDELH...,1850,0.008849,-0.048567,0.006696,...,0.023532,0.012408,-0.053530,0.001955,0.023169,-0.067590,0.104228,0.005967,0.102840,0.042471


## 4.2 测试数据

In [49]:
test = snap2020[~snap2020.seq.astype(str).isin(snap2018.seq.astype(str))]
test = test.sort_values(by='ec_number').reset_index(drop=True)
# 拼合测试数据
# uniprep = pd.read_feather('../../data/sprot_unirep_bank.feather')
test_rep =test.merge(uniprep, on='seq', how='left')

test_rep = test_rep.iloc[:, np.r_[0,2,4:7,10:1912]]
test_rep = test_rep[(test_rep.ec_specific_level>=0) | (~test_rep.isemzyme)]
test_rep.reset_index(drop=True, inplace=True)
test_rep.to_feather(cfg.TEST_FEATURE)


In [25]:
#test fasta
funclib.table2fasta(test, DATAROOT+'test.fasta')

Write finished


In [24]:
# 写入酶测试数据
test.iloc[:, np.r_[0,2,4:7,10:1912]].to_feather(cfg.TEST_FEATURE)

In [56]:
test = pd.read_feather(cfg.DATADIR+'test.feather').iloc[:,:6]

In [57]:
test

,id,isemzyme,functionCounts,ec_number,ec_specific_level,seq
0,O02324,False,0,-,0,MVSMKEFIGRWKLVHSENFEEYLKEIGVGLLIRKAASLTSPTLEIK...
1,P0DQF6,False,0,-,0,MRRFVFLAFVLVLFVIANLDSSSAQVNFSPDWGQGKRSGSSDTCAV...
2,P0DQF5,False,0,-,0,MAVILKHLAIILLVFVIEIKMGQGSQIERRSSGVTHEMLSSISKPE...
3,P0DQF4,False,0,-,0,MKIIALLLMVFLDFIIVNXAEQNEKKRRDNLGIPGAGDTEDANQAY...
4,P0DQF3,False,0,-,0,MFFLGFIIVCASEEQSDNRLPNIDFGLDRGHSGRMTAEYLMGLAAA...
...,...,...,...,...,...,...
8028,Q9NTI2,True,1,7.6.2.1,4,MLNGAGLDKALKMSLPRRSRIRSSVGPVRSSLGYKKAEDEMSRATS...
8029,D4AA47,True,1,7.6.2.1,4,MNTERDSETTFDEDSQPNDEVVPYSDDETEDELEDQGPAVEPEQNR...
8030,F1RBC8,True,1,7.6.2.4,4,MSVYSKLPSQLKKPLVKKAVVLLIALYGVKKLSPYFFGKLKGRTSK...
8031,Q7JUN3,True,1,7.6.2.4,4,MSVLSKYVDRIAEKCEHNGFTKHAFSYALVTSAILALTIKVTIPYV...


## 4.3 EC字典

In [7]:
sql="""with tb1 as (
SELECT DISTINCT ec_number FROM tb_snap_2018  UNION SELECT DISTINCT ec_number from tb_snap_2020)
SELECT DISTINCT ec_number FROM tb1 ORDER BY ec_number"""
uctools =  ucTools.ucTools('172.16.25.20')
cnx_mimic = uctools.db_conn()
res= pd.read_sql_query(sql,cnx_mimic)

In [18]:
emzyme_label_dict = {k: v for k,v in zip(res.ec_number, res.index )}
np.save(DATAROOT+'ec_label_dict.npy', emzyme_label_dict)

In [41]:
isEmzyme_dict1 = np.load(DATAROOT+'ec_label_dict.npy', allow_pickle=True).item()

In [42]:
isEmzyme_dict1.get('-')

5309

In [43]:
len(isEmzyme_dict1)

5592

In [67]:
max(isEmzyme_dict1.values())

5591

In [116]:
# 写入是几功能酶
howmany = enzyme.iloc[:,np.r_[0,4,10,12:1912]]
# howmany.to_feather(DATAROOT+'train_howmany.feather')
funclib.table2fasta(howmany, DATAROOT+'train_howmany.fasta')

Write finished


In [120]:
ectrain = train_rep.iloc[:, np.r_[0,5, 12:1912]]

In [119]:
funclib.table2fasta(ectrain, DATAROOT+'train_ec.fasta')

Write finished


In [122]:
ectrain.to_feather(DATAROOT+'train_ec.feather')

In [109]:
uctools =  ucTools.ucTools('172.16.25.20')
cnx_mimic = uctools.db_conn()
for i in range(1,8):
    sql='''with tb1 as (
                SELECT ec_number, count(ec_number) from tb_snap_2020 GROUP BY ec_number ORDER BY ec_number
                ) SELECT count(ec_number), sum(count) from tb1 WHERE ec_number like '{0}.%%';'''.format(i)
    res= pd.read_sql_query(sql,cnx_mimic)
    print(i, '\t',int(res.values[0][0]), '\t', int(res.values[0][1]))

1 	 1501 	 31265
2 	 1564 	 87931
3 	 1076 	 57991
4 	 611 	 22331
5 	 269 	 12366
6 	 200 	 25013
7 	 85 	 11495


In [ ]:
pd.read_sql_query(sql,cnx_mimic)

In [17]:
test_rep

,id,name,isemzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,P02462,CO4A1_HUMAN,False,False,0,-,0,1986-07-21,2018-05-23,2020-12-02,...,-0.010307,0.254226,0.088533,-0.009481,-0.094310,-0.155860,-0.032777,-0.141111,0.403599,0.038656
1,P00939,TPIS_RABIT,True,True,2,"5.3.1.1, 4.2.3.3",4,1986-07-21,2020-10-07,2020-12-02,...,0.380862,-0.059595,-0.008574,-0.059051,-0.067673,0.042937,-0.034098,0.182489,0.055264,0.063145
2,P02340,P53_MOUSE,False,False,0,-,0,1986-07-21,2018-10-10,2020-12-02,...,0.046348,0.070390,-0.225478,0.052106,-0.054457,-0.023183,-0.009764,-0.126373,0.027127,0.151444
3,P01848,TRAC_HUMAN,False,False,0,-,0,1986-07-21,2018-07-18,2020-08-12,...,0.034023,-0.010156,-0.237000,0.016486,-0.114827,0.272844,0.143286,0.086096,0.022759,-0.010570
4,P01733,TVBL3_HUMAN,False,False,0,-,0,1986-07-21,2018-07-18,2020-08-12,...,0.110813,0.034311,0.008858,0.002125,-0.283349,0.194242,0.050730,0.107122,0.151845,0.050503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8028,A0A061ACU2,PIEZ1_CAEEL,False,False,0,-,0,2020-12-02,2014-09-03,2020-12-02,...,0.007575,0.088760,-0.033027,0.111776,-0.303977,-0.034524,0.048425,0.022641,-0.144014,-0.013284
8029,Q8GYS8,LTG17_ARATH,False,False,0,-,0,2020-12-02,2003-03-01,2020-12-02,...,-0.039894,0.028112,0.017320,-0.007484,-0.238741,0.062047,-0.224393,0.023556,-0.157620,-0.001258
8030,Q6NRV0,TRAIP_XENLA,True,False,1,2.3.2.27,4,2020-12-02,2020-12-02,2020-12-02,...,0.010593,-0.011309,-0.014276,0.018208,-0.093581,-0.180773,0.081044,0.018184,0.379403,0.077395
8031,C5DLH0,DXO_LACTC,True,True,2,"3.6.1.-, 3.6.1.-",3,2020-12-02,2009-07-28,2020-12-02,...,0.105460,-0.045830,-0.279623,0.013364,-0.023051,-0.025641,-0.018461,-0.011174,0.054235,-0.008912


In [20]:
test_enzyme = test_isenzyme[test_rep.isemzyme]
test_enzyme.reset_index(drop=True, inplace=True)

In [21]:
test_enzyme

,id,isemzyme,f1,f2,f3,f4,f5,f6,f7,f8,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,P00939,True,0.063588,0.601079,0.325486,-0.085601,-0.253584,-0.036647,-0.029784,-0.005847,...,0.380862,-0.059595,-0.008574,-0.059051,-0.067673,0.042937,-0.034098,0.182489,0.055264,0.063145
1,P00780,True,0.007205,-0.123280,-0.034474,-0.289568,-0.647642,0.000935,0.371740,-0.028273,...,0.085781,-0.273059,-0.221449,-0.223586,-0.606882,0.184678,-0.018794,0.002871,-0.130372,-0.004079
2,P11509,True,0.001756,-0.083595,-0.034843,-0.033445,-0.026055,-0.021621,-0.011789,-0.176618,...,0.008384,0.032996,0.008304,-0.134407,-0.874854,0.438699,-0.004966,0.038901,-0.160794,0.087475
3,P03318,True,-0.018868,-0.001680,0.023652,-0.000368,0.458131,0.078176,-0.093273,-0.005638,...,0.002054,0.660758,-0.310641,0.056559,-0.505239,-0.019711,0.189862,-0.019588,-0.032944,-0.000018
4,P03317,True,-0.000369,-0.062202,0.053365,-0.001488,0.568943,0.005530,-0.312891,-0.002194,...,0.016664,0.175515,-0.057868,0.044472,-0.478046,-0.031133,-0.013975,0.016027,-0.055946,-0.040853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3574,A0A2R6Q324,True,0.000103,-0.002359,0.043916,0.022434,-0.197716,0.009035,0.105923,-0.008771,...,0.039002,0.095009,0.038163,0.036562,-0.561948,-0.003115,0.278384,0.010972,-0.301565,-0.044138
3575,Q6VE93,True,0.064012,-0.023088,0.024043,-0.005472,0.112231,-0.010082,0.283041,-0.002689,...,0.033600,-0.012503,-0.141233,0.047683,-0.082917,-0.016987,-0.001321,-0.037108,0.026485,-0.046931
3576,A0A509AKI1,True,0.009049,-0.012200,-0.043129,-0.001106,-0.037016,0.009201,-0.680708,-0.004776,...,0.032895,0.016874,-0.038663,0.039172,0.090116,-0.042056,-0.042705,0.006708,0.570306,0.161003
3577,Q6NRV0,True,0.001207,-0.053773,0.025933,-0.003389,0.003366,0.015046,-0.077923,-0.004771,...,0.010593,-0.011309,-0.014276,0.018208,-0.093581,-0.180773,0.081044,0.018184,0.379403,0.077395


In [16]:
test[test.id=='A0A059TC02']

,id,isemzyme,functionCounts,ec_number,f1,f2,f3,f4,f5,f6,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
8017,A0A059TC02,True,2,"1.2.1.44, 1.2.1.-",0.001338,-0.061641,0.004381,-0.215468,-0.198673,0.027353,...,0.066144,0.058602,-0.089901,0.023304,0.721236,-0.709499,0.05047,0.078414,0.062543,0.225015


# SECTION 5 构建EC迁移数据

In [127]:
filein = DATAROOT + 'ec_discription_20210719.txt'

In [164]:
import re

In [161]:
f= open(filein)
counter =0
reslist=[]
for line in tqdm(f):
    line = line.strip().replace('\n','')
    counter +=1
#     if counter ==1200:
#         break
    if len(line) != 0:
        if ('deleted' in line) or ('transferred' in line) or (line.count('EC')>1):
            if line.startswith('EC'):
                reslist +=[line]

8793it [00:00, 835213.33it/s]


In [173]:
writer = open(DATAROOT + 'ec_discription_20210719_r1.txt', 'w')
for line in reslist:
    writer.write(line+'\n')
writer.close()

In [182]:
deleted= []
transferred =[]
other =[]
for line in reslist:
    if 'deleted' in line:
        deleted +=[line]
    elif 'transferred' in line:
        transferred+=[line]
    else:
        other += [line]

In [176]:
len(deleted)

326

In [177]:
len(transferred)

133

In [171]:
reslist[1:15]

['EC 1.1.1.63 transferred, now EC 1.1.1.239',
 'EC 1.1.1.68 now EC 1.7.99.5',
 'EC 1.1.1.70 deleted, included in EC 1.2.1.3',
 'EC 1.1.1.74 deleted',
 'EC 1.1.1.89 deleted, included in EC 1.1.1.86',
 'EC 1.1.1.109 now EC 1.3.1.28',
 'EC 1.1.1.128 deleted covered by EC 1.1.1.264',
 'EC 1.1.1.139 deleted, included in EC 1.1.1.21',
 'EC 1.1.1.155 identical to EC 1.1.1.87',
 'EC 1.1.1.158 transferred now EC 1.3.1.98',
 'EC 1.1.1.161 deleted, covered by EC 1.14.13.15',
 'EC 1.1.1.171 now EC 1.5.1.20',
 'EC 1.1.1.180 deleted, included in EC 1.1.1.131',
 'EC 1.1.1.182 deleted, included in EC 1.1.1.198, EC 1.1.1.227 and EC 1.1.1.228']

In [ ]:
w

In [169]:
translist=[]
for item in reslist:
    if 'transferred' in item:
        subres = re.findall(r'EC[ ]*[0-9.]*', item ) +['transferred']
        translist +=[subres]

In [166]:
re.findall(r'EC[ ]*[0-9.]*', 'EC 1.1.1.63 transferred, now EC 1.1.1.239' )

['EC 1.1.1.63', 'EC 1.1.1.239']

In [61]:
train.iloc[240:255,:]

,id,isemzyme,functionCounts,ec_number,f1,f2,f3,f4,f5,f6,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
240,P03141,False,0,-,0.001659,0.005037,0.034240,-0.009932,0.301492,0.131113,...,-0.000538,-0.160595,-0.199464,-0.029431,-0.047798,-0.044202,0.274341,0.044814,-0.131824,-0.010497
241,P03140,False,0,-,0.000867,0.004244,0.026810,-0.006595,0.269940,0.103676,...,-0.001108,-0.133161,-0.121784,-0.021571,-0.039129,-0.261838,0.208429,0.041954,-0.073013,-0.010056
242,P03139,False,0,-,0.003820,0.008146,0.048195,-0.006970,0.254542,0.133930,...,-0.000376,-0.060732,-0.337160,-0.033123,-0.031427,0.050323,0.087816,0.097321,-0.095659,-0.014205
243,P03138,False,0,-,0.002133,0.004716,0.040480,-0.009079,0.333546,0.147972,...,-0.000554,-0.100190,-0.273700,-0.038758,-0.038821,0.014683,0.194105,0.041437,-0.127840,-0.011162
244,P03143,False,0,-,0.000884,0.000953,0.022431,-0.012100,0.185756,0.151870,...,-0.002725,-0.100221,-0.082165,-0.012705,-0.044696,-0.352842,0.122170,0.029388,-0.088356,-0.014819
245,P01103,False,0,-,0.001550,-0.010215,0.084963,-0.002161,-0.106861,0.024240,...,0.014058,-0.139779,-0.051309,0.023750,0.076201,-0.027757,0.444871,0.005618,0.087728,0.076237
246,P01104,False,0,-,0.007767,-0.011116,0.033822,-0.000620,-0.116797,0.008301,...,0.009997,0.014019,-0.052277,0.016994,0.027973,-0.029086,-0.031344,0.003270,0.109752,0.123043
247,P01105,False,0,-,0.003333,-0.039282,0.089289,-0.002111,-0.176063,0.009939,...,-0.009000,0.154131,-0.038165,0.008573,0.262453,-0.056291,-0.011338,0.002546,0.066857,-0.044863
248,P00959,True,1,6.1.1.10,0.000385,0.027466,0.048272,-0.124654,0.316184,0.002738,...,-0.074817,0.015226,-0.074829,0.232996,-0.182579,0.062484,0.017667,0.001724,-0.112301,-0.012241
249,P00348,True,1,1.1.1.35,0.059642,-0.236925,-0.050564,-0.107688,-0.017693,0.040009,...,-0.007338,0.211365,-0.085157,0.259117,-0.191403,0.006713,0.001691,-0.287444,-0.011161,0.045252


In [62]:
aaa = pd.read_csv(DATAROOT+'slice_train_y.txt', skiprows=1, header=None, sep=':')

In [53]:
len(set(aaa[1]))

1

In [1]:
! python ../../baselines/deepec/deepec.py -i ../../data/benchmark/data/test.fasta -o ../../data/benchmark/results/deepec

2021-07-26 16:30:00.081337: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-26 16:38:10.396227: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-26 16:38:10.436460: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.54GiB deviceMemoryBandwidth: 715.34GiB/s
2021-07-26 16:38:10.438204: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:73:00.0 name: RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.54GiB deviceMemoryBandwidth: 715.34GiB/s
2021-07-26 16:38:10.438281: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-26 16:38:10

In [233]:
test = pd.read_feather('../../data/benchmark/data/test.feather').iloc[:,:4]
train = pd.read_feather('../../data/benchmark/data/train.feather').iloc[:,:4]

In [234]:
test

,id,isemzyme,functionCounts,ec_number
0,P69031,False,0,-
1,P69019,False,0,-
2,Q5RF96,True,1,3.4.-.-
3,Q9UM00,False,0,-
4,P0C250,False,0,-
...,...,...,...,...
7096,A0A061ACU2,False,0,-
7097,Q8GYS8,False,0,-
7098,Q6NRV0,True,1,2.3.2.27
7099,C5DLH0,True,2,"3.6.1.-, 3.6.1.-"


In [60]:
sys.path.append("../../")
import benchmark_common as bcommon

In [354]:
RESULTSDIR = r'''/home/shizhenkun/codebase/BioUniprot/data/benchmark/results/'''
res_table_head = ['id', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
blast_data = pd.read_csv(RESULTSDIR+'test_blast_res.tsv', sep='\t', names=res_table_head)
data_test_blast_res = blast_data[['id', 'sseqid']].merge(train, left_on='sseqid', right_on='id', how='left').iloc[:,np.r_[0,3:6]]

In [506]:
RESULTSDIR = r'''/home/shizhenkun/codebase/BioUniprot/data/benchmark/results/'''
results = pd.read_csv(RESULTSDIR+'slice_pred.tsv', sep='\t')
results['is_enzyme_i'] = results.apply(lambda x: int(x.isemzyme_blast) if str(x.isemzyme_blast)!='nan' else x.isEnzyme_pred_xg, axis=1)
for i in range(20):
    results['top'+str(i)] = results.apply(lambda x: '' if x.is_enzyme_i==0 else x['top'+str(i)], axis=1)
results['top0'] = results.apply(lambda x: x.ec_number_blast if str(x.ec_number_blast)!='nan' else x.top0, axis=1)

for i in range(1,20):
    results['top'+str(i)] = results.apply(lambda x: '' if str(x.ec_number_blast)!='nan' else x['top'+str(i)], axis=1)
    
results['top0']=results['top0'].apply(lambda x: '' if x=='-' else x)

In [507]:
# deepec 结果
deepec_results = pd.read_csv(RESULTSDIR+'./deepec/DeepEC_Result.txt', sep='\t',names=['id', 'ec_number_deepec'], header=0) #读入文件
deepec_results.ec_number_deepec=deepec_results.apply(lambda x: x['ec_number_deepec'].replace('EC:',''), axis=1)

In [508]:
results = results.merge(deepec_results, how='left', on='id')
results = results.drop_duplicates(subset=['id'], keep='first', inplace=False)
# results = results.iloc[:,np.r_[0:5,21:26]]
results.reset_index(drop=True, inplace=True)
# results=results.merge(test, on='id', how='left')

In [509]:
results=results.merge(test.iloc[:,0:6], on='id', how='left')

In [510]:
results.to_excel(RESULTSDIR+'test_pred2.xlsx', index=None)

In [437]:
ecpred_results = pd.read_csv(RESULTSDIR+'./deepec/DeepEC_Result.txt', sep='\t', header=0) #读入文件
ecpred_results = ecpred_results.drop_duplicates(subset=['Protein ID'], keep='first')
ecpred_results = ecpred_results.rename(columns={'Protein ID':'id', 'EC Number':'ecpred_ec_number'})
ecpred_results =  ecpred_results.iloc[:,0:2]

,id,isemzyme,functionCounts,ec_number
0,P69031,False,0,-
1,P69019,False,0,-
2,Q5RF96,True,1,3.4.-.-
3,Q9UM00,False,0,-
4,P0C250,False,0,-
...,...,...,...,...
7096,A0A061ACU2,False,0,-
7097,Q8GYS8,False,0,-
7098,Q6NRV0,True,1,2.3.2.27
7099,C5DLH0,True,2,"3.6.1.-, 3.6.1.-"


In [ ]:
cccd=results[(results.is_enzyme_i==1) & results.ec_number_blast.isnull()]

In [534]:
results[(results.isemzyme) &(results.ec_number==results.top0) &(results.functionCounts>1)]

,id,top0,top1,top2,top3,top4,top5,top6,top7,top8,...,isemzyme_blast,functionCounts_blast,ec_number_blast,is_enzyme_i,ec_number_deepec,name,isemzyme,isMultiFunctional,functionCounts,ec_number
353,S0DS59,"2.3.1.-, 6.3.2.-",,,,,,,,,...,True,2.0,"2.3.1.-, 6.3.2.-",1,6.2.1.3,EQXS_GIBF5,True,True,2,"2.3.1.-, 6.3.2.-"
543,F1P963,"3.6.1.55, 3.6.1.56",,,,,,,,,...,True,2.0,"3.6.1.55, 3.6.1.56",1,NaN,8ODP_CANLF,True,True,2,"3.6.1.55, 3.6.1.56"
616,Q5AV00,"2.3.1.86, 1.1.1.100, 2.3.1.41",,,,,,,,,...,True,3.0,"2.3.1.86, 1.1.1.100, 2.3.1.41",1,2.3.1.86,ATNF_EMENI,True,True,3,"2.3.1.86, 1.1.1.100, 2.3.1.41"
647,Q5AV07,"2.3.1.86, 4.2.1.59, 1.3.1.9, 2.3.1.38, 2.3.1.3...",,,,,,,,,...,True,6.0,"2.3.1.86, 4.2.1.59, 1.3.1.9, 2.3.1.38, 2.3.1.3...",1,NaN,ATNM_EMENI,True,True,6,"2.3.1.86, 4.2.1.59, 1.3.1.9, 2.3.1.38, 2.3.1.3..."
650,Q56RZ7,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10",,,,,,,,,...,True,4.0,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10",1,NaN,LTMG_EPIFI,True,True,4,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6931,P9WEX9,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10",,,,,,,,,...,True,4.0,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10",1,2.5.1.29,DPMAD_METAN,True,True,4,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10"
7018,P9WEX3,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10",,,,,,,,,...,True,4.0,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10",1,2.5.1.29,DPASD_ARTSH,True,True,4,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10"
7034,H1VQB1,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10",,,,,,,,,...,True,4.0,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10",1,2.5.1.29,DPCHD_COLHI,True,True,4,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10"
7050,I1RL14,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10",,,,,,,,,...,True,4.0,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10",1,2.5.1.29,DPFGD_GIBZE,True,True,4,"2.5.1.-, 2.5.1.1, 2.5.1.29, 2.5.1.10"


In [484]:
results

,id,top0,top1,top2,top3,isEnzyme_pred_xg,isemzyme_blast,functionCounts_blast,ec_number_blast,is_enzyme_i,isemzyme,functionCounts,ec_number
0,P69031,,,,,0,False,0.0,-,0,False,0,-
1,P69019,,,,,0,False,0.0,-,0,False,0,-
2,Q5RF96,3.4.-.-,,,,0,True,1.0,3.4.-.-,1,True,1,3.4.-.-
3,Q9UM00,,,,,0,False,0.0,-,0,False,0,-
4,P0C250,,,,,0,False,0.0,-,0,False,0,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7096,A0A061ACU2,,,,,0,False,0.0,-,0,False,0,-
7097,Q8GYS8,,,,,0,NaN,NaN,NaN,0,False,0,-
7098,Q6NRV0,2.3.2.27,,,,0,True,1.0,2.3.2.27,1,True,1,2.3.2.27
7099,C5DLH0,3.6.1.-,,,,1,True,1.0,3.6.1.-,1,True,2,"3.6.1.-, 3.6.1.-"


In [362]:
len(set(aaa.id))

7101

In [186]:
fres=pd.DataFrame(results.id)
fres['is_enzyme'] = results.apply(lambda x: int(x.isemzyme_blast) if str(x.isemzyme_blast)!='nan' else x.isEnzyme_pred_xg, axis=1)

In [338]:
deepec_results[deepec_results.id.isin(results.id)]

,id,ec_number_deepec
183,E2JFG1,3.4.21.26
184,Q46085,2.7.7.7
185,Q65YX0,1.5.1.38
186,Q9LRI8,1.3.3.4
187,Q9CZN7,2.1.2.1
...,...,...
1115,A0A348B779,4.2.3.125
1116,A0A348B790,4.2.3.135
1117,B8PD44,4.2.3.91
1118,A0A068BGA5,2.3.1.232


In [171]:
aa= test.merge(fres, on='id', how='left')

In [184]:
fres.is_enzyme.value_counts()

0    4038
1    3063
Name: is_enzyme, dtype: int64

In [187]:
fres

,id,is_enzyme
0,P69031,0
1,P69019,0
2,Q5RF96,1
3,Q9UM00,0
4,P0C250,0
...,...,...
7096,A0A061ACU2,0
7097,Q8GYS8,0
7098,Q6NRV0,1
7099,C5DLH0,1


In [459]:
ec_transfer = pd.read_excel(DATAROOT +'ec_transfer_20210721.xlsx')
ec_transfer['ori_ec']= ec_transfer.ori_ec.apply(lambda x: x.strip())
ec_transfer['to_ec']= ec_transfer.to_ec.apply(lambda x: x.strip())

In [460]:
ec_transfer

,id,ori_ec,to_ec,way,entry_num,text
0,1,1.1.1.74,-,deleted,1,NaN
1,2,1.1.3.31,-,deleted,1,NaN
2,3,1.1.99.19,-,deleted,1,NaN
3,4,1.2.1.6,-,deleted,1,NaN
4,5,1.2.3.10,-,deleted,1,NaN
...,...,...,...,...,...,...
1353,1354,2.7.1.69,2.7.1.204,multi transfer,18,NaN
1354,1355,2.7.1.69,2.7.1.205,multi transfer,18,NaN
1355,1356,2.7.1.69,2.7.1.206,multi transfer,18,NaN
1356,1357,2.7.1.69,2.7.1.207,multi transfer,18,NaN


In [458]:
ec_transfer.to_excel(DATAROOT +'ec_transfer_20210721.xlsx', index=None)

In [420]:
translist = list(set(ec_transfer.ori_ec) & set(results.top1))

In [455]:
results[results.top0.astype('str').isin(translist)]

,id,top0,top1,top2,top3,isEnzyme_pred_xg,isemzyme_blast,functionCounts_blast,ec_number_blast,is_enzyme_i,isemzyme,functionCounts,ec_number
266,B7J3C9,3.6.3.6,3.6.3.7,5.1.3.8,3.6.3.8,1,NaN,NaN,NaN,1,True,1,3.12.1.-
1418,Q0P8J6,1.14.11.19,2.1.1.221,2.7.7.50,1.3.7.2,1,NaN,NaN,NaN,1,True,1,2.7.3.13
1753,A9QNE7,1.14.13.93,,,,1,True,1.0,1.14.13.93,1,True,1,1.14.14.137
1755,K4CI52,1.14.13.93,,,,1,True,1.0,1.14.13.93,1,True,1,1.14.14.137
1893,Q6Z7I3,1.9.3.1,5.4.99.5,5.3.99.2,4.1.1.3,1,NaN,NaN,NaN,1,True,1,2.4.2.-
2500,W8JIS9,1.14.13.89,,,,1,True,1.0,1.14.13.89,1,True,1,1.14.-.-
2549,I1TEM0,1.14.13.120,,,,1,True,1.0,1.14.13.120,1,True,1,1.14.14.-
2860,A7XXC1,1.6.5.3,2.4.2.18,6.3.1.12,2.3.2.6,1,NaN,NaN,NaN,1,False,0,-
3216,A7XXB9,1.14.11.19,3.4.-.-,5.2.1.8,1.6.5.-,1,NaN,NaN,NaN,1,False,0,-
3519,A0A0L0P4F8,5.99.1.2,,,,0,True,1.0,5.99.1.2,1,True,1,5.6.2.1


In [436]:
test[test.id=='X2EVQ5']

,id,isemzyme,functionCounts,ec_number
4258,X2EVQ5,False,0,-


In [434]:
snap2020[snap2020.id=='B8NFL5']

,id,name,isemzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
473601,B8NFL5,CP51B_ASPFN,True,False,1,1.14.14.154,4,2020-02-26,2020-02-26,2020-12-02,MGILAVILDSVCERCSGSSLWMLSTVALLSILVVSVVINVLRQLLF...,491


In [461]:
translist

['5.99.1.2',
 '3.6.3.1',
 '1.14.13.143',
 '3.6.3.54',
 '1.14.13.70',
 '1.9.3.1',
 '3.6.3.40',
 '3.6.3.6',
 '1.14.11.19',
 '3.6.3.31',
 '3.6.3.24',
 '3.6.3.7',
 '1.6.5.3',
 '1.14.13.158',
 '1.14.13.205',
 '1.14.13.120',
 '1.14.13.71',
 '3.6.3.2',
 '3.1.27.10',
 '1.14.13.93',
 '1.14.13.89']

In [514]:
len(translist)

21

In [463]:
test[~test.isemzyme]

,id,isemzyme,functionCounts,ec_number
0,P69031,False,0,-
1,P69019,False,0,-
3,Q9UM00,False,0,-
4,P0C250,False,0,-
5,Q5RBK4,False,0,-
...,...,...,...,...
7094,Q94AX3,False,0,-
7095,Q6YZW2,False,0,-
7096,A0A061ACU2,False,0,-
7097,Q8GYS8,False,0,-


In [524]:
results[(results.isemzyme)&(results.ec_number == results.top0)]

,id,top0,top1,top2,top3,top4,top5,top6,top7,top8,...,isemzyme_blast,functionCounts_blast,ec_number_blast,is_enzyme_i,ec_number_deepec,name,isemzyme,isMultiFunctional,functionCounts,ec_number
2,Q5RF96,3.4.-.-,,,,,,,,,...,True,1.0,3.4.-.-,1,NaN,SPCS1_PONAB,True,False,1,3.4.-.-
7,P9WIA9,3.1.4.3,,,,,,,,,...,True,1.0,3.1.4.3,1,NaN,PHLD_MYCTU,True,False,1,3.1.4.3
9,H2E7Q7,3.4.21.26,,,,,,,,,...,True,1.0,3.4.21.26,1,NaN,POPA_GALM3,True,False,1,3.4.21.26
12,E2JFG2,3.4.21.26,,,,,,,,,...,True,1.0,3.4.21.26,1,NaN,POPB_AMABI,True,False,1,3.4.21.26
20,H2E7Q8,3.4.21.26,,,,,,,,,...,True,1.0,3.4.21.26,1,NaN,POPB_GALM3,True,False,1,3.4.21.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7072,D6MZJ6,3.1.4.12,,,,,,,,,...,True,1.0,3.1.4.12,1,NaN,NSMA5_MOUSE,True,False,1,3.1.4.12
7074,A0A348B790,4.2.3.135,,,,,,,,,...,True,1.0,4.2.3.135,1,4.2.3.135,STS18_POSPM,True,False,1,4.2.3.135
7078,E9F5E7,2.3.1.-,,,,,,,,,...,True,1.0,2.3.1.-,1,NaN,SUBA_METRA,True,False,1,2.3.1.-
7080,Q8I4V8,5.2.1.8,,,,,,,,,...,True,1.0,5.2.1.8,1,NaN,FKB35_PLAF7,True,False,1,5.2.1.8


In [84]:
train

,id,isemzyme,functionCounts,ec_number,f1,f2,f3,f4,f5,f6,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,A0A023GPI8,False,0,-,0.000115,0.105367,0.037413,-0.004680,-0.858723,-0.005938,...,0.052682,-0.020702,0.081852,0.008791,-0.036727,-0.027552,-0.254128,0.020778,0.140904,0.140635
1,A0A023GPJ0,False,0,-,0.001688,-0.017209,0.029518,-0.006667,-0.059617,0.023538,...,0.040570,-0.031992,-0.174783,0.000952,-0.136715,0.042773,0.132279,0.015899,0.152930,-0.003412
2,A0A023PXA5,False,0,-,0.003066,-0.010453,0.013398,-0.004610,0.032372,0.012028,...,0.014996,0.034800,-0.221564,0.014209,-0.019194,-0.006493,0.121753,0.040914,0.143036,-0.024102
3,A0A023PXB0,False,0,-,0.016568,-0.021448,0.025598,-0.003162,-0.186159,0.047845,...,0.081018,-0.162543,-0.121671,0.006464,-0.045551,-0.033775,0.038735,-0.108828,0.042148,0.042277
4,A0A023PXB5,False,0,-,0.002214,-0.008754,0.012546,-0.001072,0.001833,0.001680,...,0.000570,-0.177887,-0.351524,0.017807,0.052179,0.050745,0.439712,0.071341,0.324028,-0.041494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469124,X5IWS1,False,0,-,0.000362,-0.116292,0.106362,-0.002374,-0.431054,0.029109,...,-0.043532,-0.140887,0.032296,-0.025618,-0.397847,0.008253,-0.271723,-0.034782,-0.209635,0.154475
469125,X5JA13,False,0,-,0.000235,0.023418,0.002429,-0.024888,0.715104,0.003398,...,0.000126,0.023057,-0.052336,0.024291,-0.072905,-0.127662,0.120465,0.004537,-0.033176,0.015282
469126,X5JB51,False,0,-,0.000265,0.024664,0.002740,-0.026059,0.714850,0.003718,...,0.000863,0.025310,-0.059424,0.025009,-0.076028,-0.131405,0.114073,0.005262,-0.034246,0.000684
469127,X5M5N0,True,1,2.7.11.1,0.008849,-0.048567,0.006696,-0.000739,0.013892,0.023365,...,0.023532,0.012408,-0.053530,0.001955,0.023169,-0.067590,0.104228,0.005967,0.102840,0.042471


In [85]:
test

,id,isemzyme,functionCounts,ec_number,f1,f2,f3,f4,f5,f6,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,O02324,False,0,-,0.003863,-0.052859,0.042137,0.082777,0.006852,0.061861,...,0.003677,0.295393,0.069751,0.085824,-0.325688,0.018622,0.284804,0.020171,0.143791,-0.026986
1,P0DQF6,False,0,-,0.003960,0.106233,0.269317,-0.008431,-0.236003,-0.051067,...,-0.052522,0.044518,0.078692,0.077206,-0.146649,0.049620,-0.092909,-0.689101,-0.109486,-0.078056
2,P0DQF5,False,0,-,0.001338,-0.024514,-0.000362,-0.003744,0.040726,-0.036181,...,0.016334,-0.007649,-0.018213,0.017550,-0.049806,-0.110709,0.130740,0.022192,0.611540,-0.066364
3,P0DQF4,False,0,-,0.001616,-0.013548,0.092258,-0.013326,0.130460,-0.034833,...,0.257859,0.103218,0.036438,0.015952,-0.109965,-0.183556,0.275059,-0.043923,0.004205,-0.020062
4,P0DQF3,False,0,-,0.000187,-0.012674,-0.035745,-0.005135,0.065648,-0.013928,...,0.010709,-0.021322,0.105412,0.043008,-0.158066,-0.160053,0.070354,-0.003073,0.389320,-0.072875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8028,Q9NTI2,True,1,7.6.2.1,0.005596,-0.013434,0.016993,-0.000363,-0.077348,-0.031679,...,0.014816,0.005686,-0.035825,0.014562,-0.331922,-0.014392,0.058262,0.010602,-0.037202,0.170220
8029,D4AA47,True,1,7.6.2.1,0.002222,-0.045842,0.012453,-0.000695,0.022029,0.007610,...,0.032280,0.019047,-0.010349,0.023213,-0.163688,-0.053849,0.173443,0.007751,0.089689,0.116427
8030,F1RBC8,True,1,7.6.2.4,0.000588,-0.009237,0.044856,-0.006767,-0.155198,0.004808,...,0.001776,0.128297,-0.097380,0.047742,-0.041062,-0.357653,0.024547,0.031392,0.021815,0.140106
8031,Q7JUN3,True,1,7.6.2.4,0.002590,0.152473,0.049695,-0.711504,0.122392,-0.003956,...,0.002177,-0.179690,0.000880,-0.047634,-0.424014,0.578205,0.004410,0.014373,-0.116012,0.078008
